In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import data as DATADATA
import keras.saving
from keras.models import Sequential, load_model
from keras.layers import Input, LSTM, Dense
from keras.models import Model
from tensorflow.keras.optimizers import Adam


def build_uncertainty_model():
    inputs = Input(shape=(60, 1))
    x = LSTM(128, return_sequences=True)(inputs)
    x = LSTM(64, return_sequences=False)(x)
    x = Dense(25, activation='relu')(x)
    outputs = Dense(2)(x)  # [mean, log_variance]
    return Model(inputs, outputs)

from keras.models import clone_model
from sklearn.preprocessing import MinMaxScaler
import os
import tensorflow as tf

@keras.saving.register_keras_serializable()
def gaussian_nll(y_true, y_pred):
    mean = y_pred[:, 0]
    log_var = y_pred[:, 1]
    precision = tf.exp(-log_var)
    return tf.reduce_mean(0.5 * (log_var + tf.square(y_true - mean) * precision))


os.makedirs('models-KaggleSMA', exist_ok=True)

tickers = ['SOLUSDT', 'BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'DOGEUSDT',
           'ADAUSDT', 'ALGOUSDT', 'ATOMUSDT', 'AVAXUSDT', 'BCHUSDT',
           'DOTUSDT', 'EOSUSDT', 'LINKUSDT', 'LTCUSDT', 'MATICUSDT',
           'NEOUSDT', 'PEPEUSDT', 'UNIUSDT', 'XLMUSDT', 'TUSDT']
scalers = {}  # Save scalers per ticker for later testing

# Step 1: Build combined training dataset
combined_x_train = []
combined_y_train = []

for ticker in tickers:
    big_df = DATADATA.load_asset(ticker, sampling='1d')
    big_df = DATADATA.subset(big_df, start=pd.Timestamp('2022-01-01'), end=pd.Timestamp('2026-01-01'))
    big_df['Returns'] = big_df['Close'].pct_change()
    big_df['Returns'] = big_df['Returns'].rolling(window=5).median()
    big_df.dropna(inplace=True)
    big_data = big_df.filter(['Returns']).values

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler.fit(big_data)
    
    df = DATADATA.load_asset(ticker, sampling='1d')
    df = DATADATA.subset(df, start=pd.Timestamp('2022-01-01'), end=pd.Timestamp('2024-03-01'))
    df['Returns'] = df['Close'].pct_change()
    df['Returns'] = df['Returns'].rolling(window=5).median()
    df.dropna(inplace=True)
    dataset = df.filter(['Returns']).values

    scaled_data = scaler.transform(dataset)
    scalers[ticker] = scaler

    for i in range(60, len(scaled_data)):
        combined_x_train.append(scaled_data[i - 60:i, 0])
        combined_y_train.append(scaled_data[i, 0])

# Convert combined dataset to numpy arrays
combined_x_train = np.array(combined_x_train)
combined_y_train = np.array(combined_y_train)
combined_x_train = np.reshape(combined_x_train, (combined_x_train.shape[0], combined_x_train.shape[1], 1))

# Step 2: Train base model
base_model = build_uncertainty_model()
base_model.compile(optimizer='adam', loss=gaussian_nll)
base_model.fit(combined_x_train, combined_y_train, batch_size=8, epochs=16)
base_model.save('models-KaggleSMA/base_model_uncertainty_median.keras')


Epoch 1/16
 789/1747 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - loss: -1.4272

KeyboardInterrupt: 

In [7]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
base_model = load_model('models-KaggleSMA/base_model_uncertainty_median.keras')

tickers = ['SOLUSDT', 'BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'DOGEUSDT']

# Step 3: Fine-tune separate model per ticker
for ticker in tickers:
    df = DATADATA.load_asset(ticker, sampling='1d')
    df = DATADATA.subset(df, start=pd.Timestamp('2022-01-01'), end=pd.Timestamp('2024-03-01'))
    df['Returns'] = df['Close'].pct_change()
    df['Returns'] = df['Returns'].rolling(window=5).median()
    df.dropna(inplace=True)

    data = df.filter(['Returns'])
    dataset = data.values

    scaler = scalers[ticker]
    scaled_data = scaler.transform(dataset)

    x_train = []
    y_train = []
    for i in range(60, len(scaled_data)):
        x_train.append(scaled_data[i - 60:i, 0])
        y_train.append(scaled_data[i, 0])

    x_train, y_train = np.array(x_train), np.array(y_train)
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

    # Clone base model architecture and load weights
    fine_tuned_model = clone_model(base_model)
    fine_tuned_model.set_weights(base_model.get_weights())

    # Freeze base layers
    for layer in fine_tuned_model.layers:
        layer.trainable = False

    # Add a new trainable output layer
    model = build_uncertainty_model()
    model.set_weights(base_model.get_weights())
    model.compile(optimizer='adam', loss=gaussian_nll)
    for layer in model.layers[:-1]:
        layer.trainable = False

    model.compile(optimizer=Adam(1e-3), loss=gaussian_nll)
    model.fit(x_train, y_train, epochs=8)

    # Phase 2: unfreeze base layers but use lower LR
    for layer in model.layers:
        layer.trainable = True

    model.compile(optimizer=Adam(1e-4), loss=gaussian_nll)
    model.fit(x_train, y_train, epochs=8)

    model.save(f'models-KaggleSMA/lstm_uncertainty_variable_model_{ticker}.keras')

    # Predict mean and stddev
    preds = model.predict(x_train)
    pred_mean = preds[:, 0]
    pred_std = np.sqrt(np.exp(preds[:, 1]))

    pred_mean_rescaled = scaler.inverse_transform(pred_mean.reshape(-1, 1))

    valid = data[60:].copy()
    valid['PredictedMean'] = pred_mean_rescaled
    valid['PredictedStd'] = pred_std
    valid['Z-Score'] = (valid['Returns'] - valid['PredictedMean']) / valid['PredictedStd']
    display(valid.head(10))



Epoch 1/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: -2.1759
Epoch 2/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: -2.0806
Epoch 3/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: -2.1558
Epoch 4/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: -2.1358
Epoch 5/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: -2.2106
Epoch 6/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: -2.1141
Epoch 7/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: -2.1720
Epoch 8/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: -2.1486
Epoch 1/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: -2.2226
Epoch 2/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: -2.2610
Epoch 3/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: -2.1180
Epoch 4/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: -2.0843
Epoch 5/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: -2.1774
Epoch 6/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: -2.1868
Epoch 7/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: -2.2382
Epoch 8/8


,Returns,PredictedMean,PredictedStd,Z-Score
Open time,,,,
2022-03-07,-0.050258,-0.046186,0.090839,-0.044829
2022-03-08,-0.034409,-0.049456,0.080379,0.187199
2022-03-09,0.008082,-0.024851,0.083834,0.392845
2022-03-10,-0.034409,-0.006717,0.088724,-0.312120
2022-03-11,-0.026548,-0.000650,0.107124,-0.241756
2022-03-12,0.008082,-0.004516,0.064504,0.195312
2022-03-13,-0.026548,-0.014078,0.076612,-0.162762
2022-03-14,-0.026548,-0.029774,0.052553,0.061397
2022-03-15,0.008801,-0.011632,0.062573,0.326558


Epoch 1/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: -2.3602
Epoch 2/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: -2.2954
Epoch 3/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: -2.2709
Epoch 4/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: -2.2932
Epoch 5/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: -2.3596
Epoch 6/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: -2.3401
Epoch 7/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: -2.2951
Epoch 8/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: -2.2584
Epoch 1/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: -2.3174
Epoch 2/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: -2.2962
Epoch 3/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: -2.3905
Epoch 4/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: -2.3394
Epoch 5/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: -2.3398
Epoch 6/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: -2.4334
Epoch 7/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: -2.4085
Epoch 8/8


,Returns,PredictedMean,PredictedStd,Z-Score
Open time,,,,
2022-03-07,-0.024802,-0.020837,0.099702,-0.039773
2022-03-08,-0.011265,-0.015293,0.107802,0.037366
2022-03-09,0.006368,-0.008604,0.112028,0.133645
2022-03-10,-0.011265,-0.003126,0.081415,-0.099972
2022-03-11,-0.011265,0.001524,0.079065,-0.161750
2022-03-12,0.002009,0.000087,0.062829,0.030587
2022-03-13,-0.017565,-0.002114,0.077338,-0.199781
2022-03-14,-0.017565,-0.015458,0.072681,-0.028984
2022-03-15,-0.009857,-0.008785,0.066774,-0.016053


Epoch 1/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: -2.2267
Epoch 2/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: -2.3085
Epoch 3/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: -2.3163
Epoch 4/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: -2.2865
Epoch 5/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: -2.3296
Epoch 6/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: -2.3502
Epoch 7/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: -2.3071
Epoch 8/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: -2.3110
Epoch 1/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: -2.3592
Epoch 2/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: -2.3062
Epoch 3/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: -2.3384
Epoch 4/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: -2.3750
Epoch 5/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: -2.3127
Epoch 6/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: -2.3010
Epoch 7/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: -2.3370
Epoch 8/8


,Returns,PredictedMean,PredictedStd,Z-Score
Open time,,,,
2022-03-07,-0.038445,-0.030721,0.103753,-0.074448
2022-03-08,-0.023584,-0.027982,0.100563,0.043728
2022-03-09,0.016418,-0.015258,0.123148,0.257225
2022-03-10,-0.023584,0.004630,0.104511,-0.269963
2022-03-11,-0.019120,0.004662,0.129419,-0.183755
2022-03-12,0.004670,0.002588,0.072106,0.028865
2022-03-13,-0.019120,-0.011897,0.081858,-0.088243
2022-03-14,-0.019120,-0.020262,0.048580,0.023500
2022-03-15,0.004670,-0.007201,0.065525,0.181163


Epoch 1/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: -2.5389
Epoch 2/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: -2.6987
Epoch 3/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: -2.7087
Epoch 4/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: -2.7032
Epoch 5/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: -2.6264
Epoch 6/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: -2.6997
Epoch 7/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: -2.7080
Epoch 8/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: -2.7294
Epoch 1/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: -2.7399
Epoch 2/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: -2.8054
Epoch 3/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: -2.7524
Epoch 4/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: -2.8066
Epoch 5/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: -2.7857
Epoch 6/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: -2.7800
Epoch 7/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: -2.7024
Epoch 8/8


,Returns,PredictedMean,PredictedStd,Z-Score
Open time,,,,
2022-03-07,-0.022375,-0.014302,0.054963,-0.146891
2022-03-08,-0.007447,-0.014157,0.053173,0.126196
2022-03-09,0.001389,-0.002772,0.046612,0.089287
2022-03-10,-0.007447,-0.001033,0.038994,-0.164482
2022-03-11,0.001389,-0.004012,0.042380,0.127445
2022-03-12,0.001389,0.002984,0.036047,-0.044226
2022-03-13,-0.021422,-0.000864,0.034572,-0.594660
2022-03-14,-0.021422,-0.018950,0.040810,-0.060571
2022-03-15,-0.011745,-0.013051,0.037177,0.035125


Epoch 1/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: -2.3219
Epoch 2/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: -2.2944
Epoch 3/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: -2.3484
Epoch 4/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: -2.3133
Epoch 5/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: -2.3276
Epoch 6/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: -2.3232
Epoch 7/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: -2.2302
Epoch 8/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: -2.2930
Epoch 1/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: -2.2816
Epoch 2/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: -2.3368
Epoch 3/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: -2.3338
Epoch 4/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: -2.3386
Epoch 5/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: -2.3267
Epoch 6/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: -2.3691
Epoch 7/8
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: -2.3973
Epoch 8/8


,Returns,PredictedMean,PredictedStd,Z-Score
Open time,,,,
2022-03-07,-0.033140,-0.021419,0.055566,-0.210932
2022-03-08,-0.033140,-0.030509,0.059834,-0.043974
2022-03-09,0.002571,-0.023343,0.076601,0.338288
2022-03-10,-0.033140,-0.007135,0.091058,-0.285584
2022-03-11,-0.011976,-0.010334,0.109909,-0.014943
2022-03-12,-0.006926,-0.000522,0.056345,-0.113659
2022-03-13,-0.011976,-0.008875,0.054520,-0.056870
2022-03-14,-0.011976,-0.014433,0.041221,0.059609
2022-03-15,-0.011414,-0.003778,0.053557,-0.142571
